# 데이터 전처리를 실험하는 파일
@creator: 김기범

# import

In [2]:
import pandas as pd

In [3]:
train_df = pd.read_csv('../dataset/train/train.csv')
test_df = pd.read_csv('../dataset/test/test_data.csv')

train_df.shape, test_df.shape

((32470, 6), (7765, 6))

In [4]:
train_df.head()

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32470 entries, 0 to 32469
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              32470 non-null  int64 
 1   sentence        32470 non-null  object
 2   subject_entity  32470 non-null  object
 3   object_entity   32470 non-null  object
 4   label           32470 non-null  object
 5   source          32470 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.5+ MB


## 1. Entity 분리하기

In [6]:
for column in train_df.columns:
    value = train_df.iloc[0][column]
    print(column, ':', value, type(value))

id : 0 <class 'numpy.int64'>
sentence : 〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다. <class 'str'>
subject_entity : {'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'} <class 'str'>
object_entity : {'word': '조지 해리슨', 'start_idx': 13, 'end_idx': 18, 'type': 'PER'} <class 'str'>
label : no_relation <class 'str'>
source : wikipedia <class 'str'>


In [7]:
eval(train_df.iloc[0]['subject_entity'])

{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}

In [8]:
for type_entity in ['subject', 'object']:
    column = f"{type_entity}_entity"

    word_list, type_list = [], []
    start_idx_list, end_idx_list = [], []
    
    for i in range(len(train_df)):
        dictionary = eval(train_df.iloc[i][column])

        word_list.append(dictionary['word'])
        start_idx_list.append(dictionary['start_idx'])
        end_idx_list.append(dictionary['end_idx'])
        type_list.append(dictionary['type'])
    
    train_df[column] = word_list
    for key in ['start_idx', 'end_idx', 'type']:
        train_df[f"{type_entity}_{key}"] = eval(f"{key}_list")

train_df.head()

,id,sentence,subject_entity,object_entity,label,source,subject_start_idx,subject_end_idx,subject_type,object_start_idx,object_end_idx,object_type
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,비틀즈,조지 해리슨,no_relation,wikipedia,24,26,ORG,13,18,PER
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,민주평화당,대안신당,no_relation,wikitree,19,23,ORG,14,17,ORG
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,광주FC,한국프로축구연맹,org:member_of,wikitree,21,24,ORG,34,41,ORG
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,아성다이소,박정부,org:top_members/employees,wikitree,13,17,ORG,22,24,PER
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,요미우리 자이언츠,1967,no_relation,wikipedia,22,30,ORG,0,3,DAT


# 2. 중복 데이터 처리

In [9]:
train_df.shape

(32470, 12)

In [10]:
train_df.drop(train_df[train_df.duplicated(['sentence', 'subject_entity', 'object_entity', 'label'], keep='last')].index, axis=0).shape

(32424, 12)

In [11]:
train_df.drop_duplicates(['sentence', 'subject_entity', 'object_entity', 'label'], keep='first').shape

(32424, 12)

In [12]:
label_key = {
    key: 30-idx for idx, key in enumerate(train_df['label'].value_counts().index)
}

label_key

{'no_relation': 30,
 'org:top_members/employees': 29,
 'per:employee_of': 28,
 'per:title': 27,
 'org:member_of': 26,
 'org:alternate_names': 25,
 'per:origin': 24,
 'org:place_of_headquarters': 23,
 'per:date_of_birth': 22,
 'per:alternate_names': 21,
 'per:spouse': 20,
 'per:colleagues': 19,
 'per:parents': 18,
 'org:founded': 17,
 'org:members': 16,
 'per:date_of_death': 15,
 'org:product': 14,
 'per:children': 13,
 'per:place_of_residence': 12,
 'per:other_family': 11,
 'per:place_of_birth': 10,
 'org:founded_by': 9,
 'per:product': 8,
 'per:siblings': 7,
 'org:political/religious_affiliation': 6,
 'per:religion': 5,
 'per:schools_attended': 4,
 'org:dissolved': 3,
 'org:number_of_employees/members': 2,
 'per:place_of_death': 1}

In [13]:
case_2_df = train_df[train_df.duplicated(['sentence', 'subject_entity', 'object_entity'], keep=False)].drop(train_df[train_df.duplicated(['sentence', 'subject_entity', 'object_entity', 'label'], keep=False)].index, axis=0)
case_2_df['sort'] = case_2_df['label'].apply(lambda x: label_key[x])
case_2_df.sort_values(by='sort', inplace=True)
case_2_df

,id,sentence,subject_entity,object_entity,label,source,subject_start_idx,subject_end_idx,subject_type,object_start_idx,object_end_idx,object_type,sort
11511,11511,"영화 '버즈 오브 프레이'는 배트맨이 없는 고담시에서 할리퀸, 헌트리스, 블랙 카나...",배트맨,고담시,per:place_of_residence,wikitree,16,18,PER,24,26,LOC,12
4212,4212,한편 전라남도는 최근 확진자가 발생한 순천시와 여수시에 마스크를 각각 2만장씩 총 ...,전라남도,여수시,org:members,wikitree,3,6,ORG,26,28,LOC,16
32299,32299,배우 김병철 씨가 연기하는 정복동은 천리마마트를 망하게 하기 위해 여러 계획을 세우...,정복동,김병철,per:alternate_names,wikitree,15,17,PER,3,5,PER,21
25094,25094,한편 전라남도는 최근 확진자가 발생한 순천시와 여수시에 마스크를 각각 2만장씩 총 ...,전라남도,여수시,org:place_of_headquarters,wikitree,3,6,ORG,26,28,LOC,23
3296,3296,"이날 프로그램 공개에서는 전북영산작법보존회와 김명신‧정상희의 사제동행 판소리, 관악...",강태환,색소폰,per:title,wikitree,62,64,PER,58,60,POH,27
32107,32107,"이용빈 후보는 전)문재인 대통령후보 국민주권광주선대위 공동선대위원장, 전)20대 국...",문재인,더불어민주당,per:employee_of,wikitree,10,12,PER,134,139,ORG,28
12829,12829,대한항공은 5일 조양호 회장의 3자녀가 보유한 싸이버스카이 주식 9만9900주 전량...,대한항공,조양호,org:top_members/employees,wikipedia,0,3,ORG,9,11,PER,29
13496,13496,"2015년 11월 조현아 전 대한항공 부사장, 조원태 대한항공 부사장, 조현민 대한...",대한항공,조양호,org:top_members/employees,wikipedia,16,19,ORG,54,56,PER,29
6749,6749,대한항공은 5일 조양호 회장의 3자녀가 보유한 싸이버스카이 주식 9만9900주 전량...,대한항공,조양호,no_relation,wikipedia,0,3,ORG,9,11,PER,30
8364,8364,배우 김병철 씨가 연기하는 정복동은 천리마마트를 망하게 하기 위해 여러 계획을 세우...,정복동,김병철,no_relation,wikitree,15,17,PER,3,5,PER,30


In [14]:
case_2_df[case_2_df.duplicated(['sentence', 'subject_entity', 'object_entity'], keep='first')]

,id,sentence,subject_entity,object_entity,label,source,subject_start_idx,subject_end_idx,subject_type,object_start_idx,object_end_idx,object_type,sort
25094,25094,한편 전라남도는 최근 확진자가 발생한 순천시와 여수시에 마스크를 각각 2만장씩 총 ...,전라남도,여수시,org:place_of_headquarters,wikitree,3,6,ORG,26,28,LOC,23
6749,6749,대한항공은 5일 조양호 회장의 3자녀가 보유한 싸이버스카이 주식 9만9900주 전량...,대한항공,조양호,no_relation,wikipedia,0,3,ORG,9,11,PER,30
8364,8364,배우 김병철 씨가 연기하는 정복동은 천리마마트를 망하게 하기 위해 여러 계획을 세우...,정복동,김병철,no_relation,wikitree,15,17,PER,3,5,PER,30
10320,10320,"이용빈 후보는 전)문재인 대통령후보 국민주권광주선대위 공동선대위원장, 전)20대 국...",문재인,더불어민주당,no_relation,wikitree,10,12,PER,52,57,ORG,30
18458,18458,"2015년 11월 조현아 전 대한항공 부사장, 조원태 대한항공 부사장, 조현민 대한...",대한항공,조양호,no_relation,wikipedia,44,47,ORG,54,56,PER,30
22258,22258,"영화 '버즈 오브 프레이'는 배트맨이 없는 고담시에서 할리퀸, 헌트리스, 블랙 카나...",배트맨,고담시,no_relation,wikitree,16,18,PER,24,26,LOC,30


In [15]:
case_2_df[case_2_df.duplicated(['sentence', 'subject_entity', 'object_entity'], keep='last')]

,id,sentence,subject_entity,object_entity,label,source,subject_start_idx,subject_end_idx,subject_type,object_start_idx,object_end_idx,object_type,sort
11511,11511,"영화 '버즈 오브 프레이'는 배트맨이 없는 고담시에서 할리퀸, 헌트리스, 블랙 카나...",배트맨,고담시,per:place_of_residence,wikitree,16,18,PER,24,26,LOC,12
4212,4212,한편 전라남도는 최근 확진자가 발생한 순천시와 여수시에 마스크를 각각 2만장씩 총 ...,전라남도,여수시,org:members,wikitree,3,6,ORG,26,28,LOC,16
32299,32299,배우 김병철 씨가 연기하는 정복동은 천리마마트를 망하게 하기 위해 여러 계획을 세우...,정복동,김병철,per:alternate_names,wikitree,15,17,PER,3,5,PER,21
32107,32107,"이용빈 후보는 전)문재인 대통령후보 국민주권광주선대위 공동선대위원장, 전)20대 국...",문재인,더불어민주당,per:employee_of,wikitree,10,12,PER,134,139,ORG,28
12829,12829,대한항공은 5일 조양호 회장의 3자녀가 보유한 싸이버스카이 주식 9만9900주 전량...,대한항공,조양호,org:top_members/employees,wikipedia,0,3,ORG,9,11,PER,29
13496,13496,"2015년 11월 조현아 전 대한항공 부사장, 조원태 대한항공 부사장, 조현민 대한...",대한항공,조양호,org:top_members/employees,wikipedia,16,19,ORG,54,56,PER,29


In [16]:
case_2_df.drop_duplicates(['sentence', 'subject_entity', 'object_entity'], keep='first')

,id,sentence,subject_entity,object_entity,label,source,subject_start_idx,subject_end_idx,subject_type,object_start_idx,object_end_idx,object_type,sort
11511,11511,"영화 '버즈 오브 프레이'는 배트맨이 없는 고담시에서 할리퀸, 헌트리스, 블랙 카나...",배트맨,고담시,per:place_of_residence,wikitree,16,18,PER,24,26,LOC,12
4212,4212,한편 전라남도는 최근 확진자가 발생한 순천시와 여수시에 마스크를 각각 2만장씩 총 ...,전라남도,여수시,org:members,wikitree,3,6,ORG,26,28,LOC,16
32299,32299,배우 김병철 씨가 연기하는 정복동은 천리마마트를 망하게 하기 위해 여러 계획을 세우...,정복동,김병철,per:alternate_names,wikitree,15,17,PER,3,5,PER,21
3296,3296,"이날 프로그램 공개에서는 전북영산작법보존회와 김명신‧정상희의 사제동행 판소리, 관악...",강태환,색소폰,per:title,wikitree,62,64,PER,58,60,POH,27
32107,32107,"이용빈 후보는 전)문재인 대통령후보 국민주권광주선대위 공동선대위원장, 전)20대 국...",문재인,더불어민주당,per:employee_of,wikitree,10,12,PER,134,139,ORG,28
12829,12829,대한항공은 5일 조양호 회장의 3자녀가 보유한 싸이버스카이 주식 9만9900주 전량...,대한항공,조양호,org:top_members/employees,wikipedia,0,3,ORG,9,11,PER,29
13496,13496,"2015년 11월 조현아 전 대한항공 부사장, 조원태 대한항공 부사장, 조현민 대한...",대한항공,조양호,org:top_members/employees,wikipedia,16,19,ORG,54,56,PER,29


In [17]:
view_df = train_df.drop(case_2_df.index, axis=0) # case 2에 해당하는 데이터 삭제
case_2_df.drop_duplicates(['sentence', 'subject_entity', 'object_entity'], keep='first', inplace=True)
new_df = pd.concat([view_df, case_2_df[case_2_df.columns[:-1]]], axis=0)

train_df.shape, view_df.shape, new_df.shape

((32470, 12), (32457, 12), (32464, 12))

In [18]:
new_df.columns

Index(['id', 'sentence', 'subject_entity', 'object_entity', 'label', 'source',
       'subject_start_idx', 'subject_end_idx', 'subject_type',
       'object_start_idx', 'object_end_idx', 'object_type'],
      dtype='object')

# 3. 특수 태그 달기

- entity 사이에 [ENT] [/ENT] 추가
- 일본어와 한자는 정규식 이용해서 태그 추가 ([JAP], [CHN])

## 3.1 entity 사이에 [ENT] 태그 추가

In [19]:
for _, row in train_df.iterrows():
    sentence = row['sentence']
    print(sentence)
    for check, idx in enumerate(sorted([row['subject_start_idx'], row['subject_end_idx'], row['object_start_idx'], row['object_end_idx']], reverse=True)):
        if check % 2 == 0:
            sentence = sentence[:idx+1] + " [\ENT] " + sentence[idx+1:]
        else:
            sentence = sentence[:idx] + "[ENT] " + sentence[idx:]
    print(sentence)
    break
print("Done.")

〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
〈Something〉는 [ENT] 조지 해리슨 [\ENT] 이 쓰고 [ENT] 비틀즈 [\ENT] 가 1969년 앨범 《Abbey Road》에 담은 노래다.
Done.


### 3.1.1 entity 사이에 [{S|O}:{type}] 태그 추가

In [20]:
for _, row in train_df.iterrows():
    print("="*5, "subject", "="*5)
    print(row['subject_entity'], "-", row['subject_type'])
    print("="*5, "object", "="*5)
    print(row['object_entity'], "-", row['object_type'])

    sentence = row['sentence']
    print(sentence)
    trigger = True if row['object_end_idx'] > row['subject_end_idx'] else False
    for check, idx in enumerate(sorted([row['subject_start_idx'], row['subject_end_idx'], row['object_start_idx'], row['object_end_idx']], reverse=True)):
        if trigger:
            token = f"O:{row['object_type']}"
        else:
            token = f"S:{row['subject_type']}"

        if check % 2 == 0:
            sentence = sentence[:idx+1] + f" [\{token}] " + sentence[idx+1:]
        else:
            sentence = sentence[:idx] + f"[{token}] " + sentence[idx:]
            trigger = not trigger
    print(sentence)
    break
print("Done.")

===== subject =====
비틀즈 - ORG
===== object =====
조지 해리슨 - PER
〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
〈Something〉는 [O:PER] 조지 해리슨 [\O:PER] 이 쓰고 [S:ORG] 비틀즈 [\S:ORG] 가 1969년 앨범 《Abbey Road》에 담은 노래다.
Done.


## 3.2 [JAP], [CHN] 태그 달아주기

In [21]:
jap_idx = train_df['sentence'].str.extract('([ぁ-ゔァ-ヴー々〆〤一-龥]+)').dropna().index

jap_idx, len(jap_idx)

(Int64Index([    7,    62,    69,    83,    91,   152,   191,   211,   215,
               233,
             ...
             32318, 32347, 32361, 32381, 32393, 32402, 32411, 32438, 32449,
             32456],
            dtype='int64', length=2334),
 2334)

In [22]:
view_df = train_df.copy(deep=True)
view_df['sentence'].replace(r'[ぁ-ゔァ-ヴー々〆〤一-龥]+', '[OTH]', regex=True, inplace=True)

view_df.loc[jap_idx]

,id,sentence,subject_entity,object_entity,label,source,subject_start_idx,subject_end_idx,subject_type,object_start_idx,object_end_idx,object_type
7,7,"박용오([OTH], 1937년 4월 29일(음력 3월 19일)(음력 3월 19일) ...",박용오,1937년 4월 29일,per:date_of_birth,wikipedia,0,2,PER,9,20,DAT
62,62,"김동성([OTH], 1980년 2월 9일 ~)은 대한민국의 쇼트트랙 선수이다.",김동성,1980년 2월 9일,per:date_of_birth,wikipedia,0,2,PER,9,19,DAT
69,69,칸트는 미([OTH])란 숭고함이다라고 주장하였는데 그의 이러한 주장은 윌리엄 터너...,윌리엄 터너,낭만주의,no_relation,wikipedia,36,41,PER,78,81,POH
83,83,진도군은 진도개를 보기 위해 찾아온 관람객들에게 더욱 흥미롭고 즐거움을 선사하기 위...,진도군,진도개,no_relation,wikitree,0,2,ORG,5,7,POH
91,91,"백한성([OTH], [OTH], 1899년 6월 15일 조선 충청도 공주 출생 ~ ...",백한성,조선 충청도 공주,per:place_of_birth,wikipedia,0,2,PER,28,36,LOC
...,...,...,...,...,...,...,...,...,...,...,...,...
32402,32402,"1990년 국가보훈처에 의해 이상룡([OTH]), 이승화([OTH]), 이광민([O...",이광민,독립운동가,per:title,wikipedia,36,38,PER,49,53,POH
32411,32411,"가톨릭평화방송(가톨릭[OTH], Catholic Peace Broadcasting ...",가톨릭평화방송,가톨릭平和放送,org:alternate_names,wikipedia,0,6,ORG,8,14,ORG
32438,32438,"간 나오토는 총리 취임에 즈음해 당 간사장에 에다노 유키오, 내각관방장관에 센고쿠 ...",에다노 유키오,센고쿠 요시토,no_relation,wikipedia,25,31,PER,42,48,PER
32449,32449,"이때 만주 지역에는 크고작은 독립군이 조직되었고, 동만주지방의 홍범도가 이끄는 대한...",홍범도,대한독립군,per:employee_of,wikipedia,35,37,PER,44,48,ORG


# 4. 스폐셜 토큰 추가

In [23]:
arr = ['[OTH]', '[ENT]', '[/ENT]']

# entity detail
for category in ['S', 'O']:
    for type in ['PER', 'NOH', 'ORG', 'LOC', 'POH', 'DUR', 'PNT', 'TIM', 'MNY', 'DAT']:
        arr.append(f"[{category}:{type}]")
        arr.append(f"[/{category}:{type}]")
    
arr, len(arr)

(['[OTH]',
  '[ENT]',
  '[/ENT]',
  '[S:PER]',
  '[/S:PER]',
  '[S:NOH]',
  '[/S:NOH]',
  '[S:ORG]',
  '[/S:ORG]',
  '[S:LOC]',
  '[/S:LOC]',
  '[S:POH]',
  '[/S:POH]',
  '[S:DUR]',
  '[/S:DUR]',
  '[S:PNT]',
  '[/S:PNT]',
  '[S:TIM]',
  '[/S:TIM]',
  '[S:MNY]',
  '[/S:MNY]',
  '[S:DAT]',
  '[/S:DAT]',
  '[O:PER]',
  '[/O:PER]',
  '[O:NOH]',
  '[/O:NOH]',
  '[O:ORG]',
  '[/O:ORG]',
  '[O:LOC]',
  '[/O:LOC]',
  '[O:POH]',
  '[/O:POH]',
  '[O:DUR]',
  '[/O:DUR]',
  '[O:PNT]',
  '[/O:PNT]',
  '[O:TIM]',
  '[/O:TIM]',
  '[O:MNY]',
  '[/O:MNY]',
  '[O:DAT]',
  '[/O:DAT]'],
 43)

# 5. 불용어 제거

In [37]:
from konlpy.tag import Okt, Hannanum, Kkma

In [30]:
# 불용어 리스트
stop_words = set()
with open('../utils/stop_word.txt', 'r') as f:
    for line in f.readlines():
        stop_words.add(line.strip())

for idx, word in enumerate(stop_words):
    if idx == 10: break
    print(word, end=" ")

이 되다 할수있어 비슷하다 근거하여 그치지 않다 시각 아하 저기 가까스로 그런데 

In [36]:
# Okt: 트위터 데이터
okt = Okt()

for i in range(len(train_df)):
    if i == 10: break
    new_sentence = []
    for word in okt.morphs(train_df['sentence'][i]):
        if word not in stop_words:
           new_sentence.append(word)
    
    print("="*5, i, "="*5)
    print("original >>", train_df['sentence'][i])
    print("preprocessing >>", " ".join(new_sentence))

===== 0 =====
original >> 〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
preprocessing >> 〈 Something 〉 는 조지 해리슨 쓰고 비틀즈 1969년 앨범 《 Abbey Road 》 담은 노래 다 .
===== 1 =====
original >> 호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.
preprocessing >> 호남 기반 인 바른 미래 당 · 대안 신당 · 민주 평화 당 우여곡절 끝 합당 해 민생 당 ( 가칭 ) 재탄생 한다 .
===== 2 =====
original >> K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.
preprocessing >> K 리그 2 성적 1 위 달리 고 있는 광주 FC 는 지난 26일 한국 프로축구 연맹 으로부터 관중 유치 성과 마케팅 성과 인정받아 ‘ 풀 스타디움 상 ’ ‘ 플러스 스타디움 상 ’ 수상했다 .
===== 3 =====
original >> 균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
preprocessing >> 균일 생활용품 점 ( 주 ) 아성 다이소 ( 대표 박 정부 ) 는 코로나 19 바이러스 어려움 겪고 있는 대구광역시 행복 박스 전달 했다고 10일 밝혔다 .
===== 4 =====
original >> 1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8번으로 배정되었다.
preprocessing >> 1967년 프로 야구 드래프트 1 순위 요미우리 자이언츠 입단 번호 는 8 번 배정 되었다 .
===== 5 =====
original >> : 유엔, 유럽 의회, 북대서양 조약 기구 (NA

In [38]:
# Hannanum: 한나눔
hannanum = Hannanum()

for i in range(len(train_df)):
    if i == 10: break
    new_sentence = []
    for word in hannanum.morphs(train_df['sentence'][i]):
        if word not in stop_words:
           new_sentence.append(word)
    
    print("="*5, i, "="*5)
    print("original >>", train_df['sentence'][i])
    print("preprocessing >>", " ".join(new_sentence))

===== 0 =====
original >> 〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
preprocessing >> 〈Something〉 는 조 지 해리슨 쓰 고 비틀즈 1969년 앨범 《Abbey Road》 담 은 노래 다 .
===== 1 =====
original >> 호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.
preprocessing >> 호남 기반 ㄴ 바른미래당·대안신당·민주평화당 우여곡절 끝 합당 민생당(가칭) 재탄생 ㄴ다 .
===== 2 =====
original >> K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.
preprocessing >> K리그2 성적 1위 달리 고 있 는 광주FC 는 지나 ㄴ 26일 한국프로축구연맹 으로부터 관중 유치 성과 마케팅 성과 인정 받 ‘풀 스타디움상’ ‘플러스 스타디움상’ 수상 었다 .
===== 3 =====
original >> 균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
preprocessing >> 균일가 생활용품점 (주)아성다이소(대표 박정부 ) 는 코로나19 바이러스 어려움 겪 고 있 는 대구광역시 행복박스 전달 었다 고 10일 밝히 었다 .
===== 4 =====
original >> 1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8번으로 배정되었다.
preprocessing >> 1967년 프로 야구 드래프트 1순위 요미우리 자이언츠에 게 입단 면서 등번호 는 8번 배정 되 었다 .
===== 5 =====
original >> : 유엔, 유럽 의회, 북대서양 조약 기구 (NATO), 국제이주기구, 세계 보건 기구 

In [39]:
# Kkma: 꼬꼬마
kkma = Kkma()

for i in range(len(train_df)):
    if i == 10: break
    new_sentence = []
    for word, _ in kkma.pos(train_df['sentence'][i]):
        if word not in stop_words:
           new_sentence.append(word)
    
    print("="*5, i, "="*5)
    print("original >>", train_df['sentence'][i])
    print("preprocessing >>", " ".join(new_sentence))

===== 0 =====
original >> 〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.
preprocessing >> 〈 Something 〉 늘 ㄴ 조지 해리슨 쓰 고 비틀즈 1969 앨범 《 Abbey Road 》 담 은 노래 다 .
===== 1 =====
original >> 호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.
preprocessing >> 호남 기반 ㄴ 바른 미래 당 · 대안 신당 · 민주 평화 당 우여곡절 끝 합 당하 민생 당 ( 가칭 ) 재 탄생 ㄴ다 .
===== 2 =====
original >> K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.
preprocessing >> K 리그 2 성적 1 위 달리 고 있 는 광주 FC 는 지나 ㄴ 26 한국 프로 축구 연맹 관중 유치 성과 마케팅 성과 인정받 ‘ 풀 스타디움 상 ’ ‘ 플러스 스타디움 상 ’ 수상 었 다 .
===== 3 =====
original >> 균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.
preprocessing >> 균 일가 생활 용품 점 ( 주 ) 아성 다이 소 ( 대표 박 정부 ) 늘 ㄴ 코로나 19 바이러스 어려움 겪 고 있 는 대구광역시 행복 박스 전달하 었 다고 10 밝히 었 다 .
===== 4 =====
original >> 1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8번으로 배정되었다.
preprocessing >> 1967 프로 야구 드래프트 1 순위 요 미 자이언츠 입단 면서 번호 는 8 번 배정 되 었 다 .
===== 5 =====
original >> : 유엔, 유럽 

# 6. 띄어쓰기 교정